In [1]:
print("hi")

hi


In [ ]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf

# ---------------- CONFIG ----------------
IMG_SIZE = 100
DATASET_DIR = "dataset"
MODEL_PATH = "face_cnn.h5"
CONF_THRESHOLD = 0.75
RESET_TIME = 300  # 5 minutes

os.makedirs(DATASET_DIR, exist_ok=True)

# ---------------- FACE DETECTOR ----------------
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ---------------- LOAD MODEL ----------------
model = tf.keras.models.load_model(MODEL_PATH)

# ---------------- UTILS ----------------
def preprocess(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    face = face / 255.0
    return face.reshape(1, IMG_SIZE, IMG_SIZE, 1)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def load_embeddings():
    embeddings = {}
    for person in os.listdir(DATASET_DIR):
        person_path = os.path.join(DATASET_DIR, person)
        if not os.path.isdir(person_path):
            continue

        vectors = []
        for img_name in os.listdir(person_path):
            img = cv2.imread(os.path.join(person_path, img_name), cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0
            emb = model.predict(img.reshape(1, IMG_SIZE, IMG_SIZE, 1), verbose=0)[0]
            vectors.append(emb)

        if len(vectors) > 0:
            embeddings[person] = np.mean(vectors, axis=0)

    return embeddings


# ---------------- INITIAL LOAD ----------------
known_embeddings = load_embeddings()
attendance = {}
last_reset = time.time()

# ---------------- CAMERA LOOP ----------------
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Reset attendance every 5 minutes
    if time.time() - last_reset > RESET_TIME:
        attendance.clear()
        last_reset = time.time()

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_input = preprocess(face)
        face_vec = model.predict(face_input, verbose=0)[0]

        name = "Unknown"
        best_score = 0

        for person, ref_vec in known_embeddings.items():
            score = cosine_similarity(face_vec, ref_vec)
            if score > best_score:
                best_score = score
                name = person

        if best_score < 0.75:
            name = "Unknown"

        # Mark attendance only once
        if name != "Unknown" and name not in attendance:
            attendance[name] = time.strftime("%H:%M:%S")

        # Draw UI
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, name, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.putText(
        frame,
        f"Total Present: {len(attendance)}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 0, 0),
        2
    )

    cv2.imshow("Face Attendance System", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
